# Ensemble Learning

## Initial Imports

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [33]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [19]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()
df["home_ownership_code"] = ord_enc.fit_transform(df[["home_ownership"]])
df["verification_status_code"] = ord_enc.fit_transform(df[["verification_status"]])
df["issue_d_code"] = ord_enc.fit_transform(df[["issue_d"]])
df["pymnt_plan_code"] = ord_enc.fit_transform(df[["pymnt_plan"]])
df["initial_list_status_code"] = ord_enc.fit_transform(df[["initial_list_status"]])
df["application_type_code"] = ord_enc.fit_transform(df[["application_type"]])
df["hardship_flag_code"] = ord_enc.fit_transform(df[["hardship_flag"]])
df["debt_settlement_flag_code"] = ord_enc.fit_transform(df[["debt_settlement_flag"]])
df.drop(columns=["home_ownership","verification_status","issue_d","pymnt_plan","initial_list_status","next_pymnt_d","application_type","hardship_flag","debt_settlement_flag"], inplace=True)
df.info()
#"verification_status_code","issue_d_code","pymnt_plan_code","initial_list_status","next_pymnt_d_code","application_type_code","hardship_flag_code","debt_settlement_flag"
#,"verification_status","issue_d","pymnt_plan","initial_list_status","next_pymnt_d","application_type","hardship_flag","debt_settlement_flag"
#from sklearn.preprocessing import OneHotEncoder
#oe_style = OneHotEncoder()
#oe_results = oe_style.fit_transform(obj_df[["body_style"]])
#pd.DataFrame(oe_results.toarray(), columns=oe_style.categories_).head()
#df["home_ownership"] = df["home_ownership"].astype('category')
#df["home_ownership_cat"] = df["home_ownership"].cat.codes
#df.dtypes
# Preview the data"
#df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68817 entries, 0 to 68816
Data columns (total 85 columns):
loan_amnt                     68817 non-null float64
int_rate                      68817 non-null float64
installment                   68817 non-null float64
annual_inc                    68817 non-null float64
loan_status                   68817 non-null object
dti                           68817 non-null float64
delinq_2yrs                   68817 non-null float64
inq_last_6mths                68817 non-null float64
open_acc                      68817 non-null float64
pub_rec                       68817 non-null float64
revol_bal                     68817 non-null float64
total_acc                     68817 non-null float64
out_prncp                     68817 non-null float64
out_prncp_inv                 68817 non-null float64
total_pymnt                   68817 non-null float64
total_pymnt_inv               68817 non-null float64
total_rec_prncp               68817 non-null

## Split the Data into Training and Testing

In [20]:
# Create our features
x = df.drop(columns="loan_status")
# Create our target
y = df["loan_status"]

In [21]:
x.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,total_bc_limit,total_il_high_credit_limit,home_ownership_code,verification_status_code,issue_d_code,pymnt_plan_code,initial_list_status_code,application_type_code,hardship_flag_code,debt_settlement_flag_code
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,29734.128558,5.572240e+04,1.812779,0.669994,0.805542,0.0,0.876121,0.139660,0.0,0.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,26795.394232,5.095845e+04,0.941313,0.719105,0.714932,0.0,0.329446,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,100.000000,1.270000e+02,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,11600.000000,2.288000e+04,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.0,0.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,22100.000000,4.200000e+04,1.000000,1.000000,1.000000,0.0,1.000000,0.000000,0.0,0.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,39300.000000,7.249900e+04,3.000000,1.000000,1.000000,0.0,1.000000,0.000000,0.0,0.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,509400.000000,1.426964e+06,3.000000,2.000000,2.000000,0.0,1.000000,1.000000,0.0,0.0


In [22]:
# Check the balance of our target values
print(y.value_counts())

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64


In [23]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=78)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [24]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [25]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
x_scaler = scaler.fit(x_train)

In [26]:
# Scale the training and testing data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [27]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rf_model = BalancedRandomForestClassifier(n_estimators=500, random_state=78)
rf_model.fit(x_train, y_train)
y_pred = rf_model.predict(x_test)

In [37]:
# Calculated the balanced accuracy score
#from sklearn.metrics import accuracy_score
acc_score = balanced_accuracy_score(y_test, y_pred)
print(f"Balanced Accuracy Score : {acc_score}")

Accuracy Score : 0.7303384917136476


In [30]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm= confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
print("Confusion Matrix")
display(cm_df)


Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,52,40
Actual 1,1789,15324


In [38]:
# Print the imbalanced classification report
print("Imbalanced Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Imbalanced Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.57      0.90      0.05      0.71      0.49        92
   low_risk       1.00      0.90      0.57      0.94      0.71      0.52     17113

avg / total       0.99      0.89      0.57      0.94      0.71      0.52     17205



In [39]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
sorted(zip(rf_model.feature_importances_, x.columns), reverse=True)

[(0.07482059865791764, 'total_rec_prncp'),
 (0.06220496283362564, 'total_pymnt'),
 (0.062044801204460265, 'total_rec_int'),
 (0.0617601330221155, 'last_pymnt_amnt'),
 (0.054855729942975294, 'total_pymnt_inv'),
 (0.02931323652303738, 'int_rate'),
 (0.02192322438644246, 'issue_d_code'),
 (0.019541553295698778, 'dti'),
 (0.0188940814916535, 'mo_sin_old_rev_tl_op'),
 (0.017498479964337576, 'annual_inc'),
 (0.01691779509580381, 'bc_util'),
 (0.016461262240790624, 'installment'),
 (0.016206274711504803, 'mths_since_recent_inq'),
 (0.015462268334515514, 'max_bal_bc'),
 (0.01496204938603574, 'total_bc_limit'),
 (0.014859165003720597, 'bc_open_to_buy'),
 (0.014760416286321285, 'out_prncp'),
 (0.014572274483912991, 'revol_bal'),
 (0.01447335868553449, 'out_prncp_inv'),
 (0.01438441142451517, 'mo_sin_old_il_acct'),
 (0.014030976515586486, 'mths_since_rcnt_il'),
 (0.014028404341696613, 'tot_hi_cred_lim'),
 (0.0139455163881746, 'il_util'),
 (0.013697563937877643, 'avg_cur_bal'),
 (0.012920337365994

### Easy Ensemble Classifier

In [42]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
#base_estimator = AdaBoostClassifier(n_estimators=10)
eec = EasyEnsembleClassifier(n_estimators=10)
eec.fit(x_train, y_train)
y_pred_eec = eec.predict(x_test)


In [44]:
# Calculated the balanced accuracy score
acc_score_eec = balanced_accuracy_score(y_test, y_pred_eec)
print(f"Balanced Accuracy Score : {acc_score_eec}")

Balanced Accuracy Score : 0.8190782369873908


In [45]:
# Display the confusion matrix
cm_eec= confusion_matrix(y_test, y_pred_eec)
cm_df_eec = pd.DataFrame(
    cm_eec, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
print("Confusion Matrix")
display(cm_df_eec)

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,71,21
Actual 1,2286,14827


In [46]:
# Print the imbalanced classification report
print("Imbalanced Classification Report")
print(classification_report_imbalanced(y_test, y_pred_eec))

Imbalanced Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.77      0.87      0.06      0.82      0.66        92
   low_risk       1.00      0.87      0.77      0.93      0.82      0.67     17113

avg / total       0.99      0.87      0.77      0.92      0.82      0.67     17205

